In [20]:
import pandas as pd

In [21]:
import nltk
nltk.download("gutenberg")

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\vasud\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [22]:
from nltk.corpus import gutenberg
data=gutenberg.raw("milton-paradise.txt")

with open("milton-paradise.txt",'w')as file:
    file.write(data)

In [23]:
import tensorflow

In [24]:
#data pre preprosesing

import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
#load the data
with open ('milton-paradise.txt','r') as file:
    text=file.read().lower()

In [25]:
#convert into token
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])

In [35]:
#Total number of words in Text
total_word=len(tokenizer.word_index)+1
total_word

9148

In [27]:
#Create input sequence
input_seq=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_seq.append(n_gram_sequence)

In [28]:
#max sequence
max_seq_len = max(len(x) for x in input_seq)
input_sequences= np.array (pad_sequences(input_seq,maxlen=max_seq_len,padding="pre"))

In [29]:
input_sequences

array([[   0,    0,    0, ...,    0,  186,  169],
       [   0,    0,    0, ...,  186,  169,   20],
       [   0,    0,    0, ...,  169,   20, 3314],
       ...,
       [   0,    0,    0, ...,   13, 1875,  121],
       [   0,    0,    0, ...,    0,    2,  151],
       [   0,    0,    0, ...,    2,  151, 9147]])

In [30]:
x=input_sequences[:,:-1]
y=input_sequences[:,-1]

In [31]:
x

array([[   0,    0,    0, ...,    0,    0,  186],
       [   0,    0,    0, ...,    0,  186,  169],
       [   0,    0,    0, ...,  186,  169,   20],
       ...,
       [   0,    0,    0, ...,  555,   13, 1875],
       [   0,    0,    0, ...,    0,    0,    2],
       [   0,    0,    0, ...,    0,    2,  151]])

In [32]:
y

array([ 169,   20, 3314, ...,  121,  151, 9147])

In [36]:
from tensorflow.keras.utils import  to_categorical

y=to_categorical(y,num_classes=total_word)

In [37]:
print(y)
print(len(y[0]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
9148


In [39]:
# split the data into traning and testing 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [40]:
#Early stooping
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',
                            patience=3,
                            restore_best_weights=True
                            )

In [42]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Embedding,GRU,Flatten,Dropout

model= Sequential()
model.add(Embedding(total_word+1,1000,
                    input_length=max_seq_len
                    ))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_word,activation="softmax"))


d:\btch 16\NPl\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [43]:
#compile model
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['acc'])

In [44]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.fit(x_train,y_train,epochs=25,validation_data=(x_test,y_test))

Epoch 1/25
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 147s 94ms/step - acc: 0.0449 - loss: 7.2456 - val_acc: 0.0580 - val_loss: 7.0219
Epoch 2/25
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 140s 92ms/step - acc: 0.0641 - loss: 6.6801 - val_acc: 0.0652 - val_loss: 6.9680
Epoch 3/25
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 147s 96ms/step - acc: 0.0763 - loss: 6.3448 - val_acc: 0.0702 - val_loss: 7.0105
Epoch 4/25
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 147s 97ms/step - acc: 0.0883 - loss: 6.0116 - val_acc: 0.0681 - val_loss: 7.1250
Epoch 5/25
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 148s 97ms/step - acc: 0.1009 - loss: 5.6708 - val_acc: 0.0697 - val_loss: 7.2417
Epoch 6/25
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 149s 98ms/step - acc: 0.1184 - loss: 5.3177 - val_acc: 0.0681 - val_loss: 7.3970
Epoch 7/25
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 149s 98ms/step - acc: 0.1429 - loss: 4.9761 - val_acc: 0.0658 - val_loss: 7.5512
Epoch 8/25
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 149s 98ms/step - acc: 0.1744 - loss: 4.6488 - val_acc: 0.0610 - val_loss: 7.7641
Epoch 9/25
1521/

KeyboardInterrupt: 

In [ ]:
model.save("Next_word_pridection_model.h5")

In [ ]:
def generate_text(seed_text, length=20):
    for _ in range(length):
        encoded = tokenizer.texts_to_sequences([seed_text[-max_seq_len:]])[0]
        encoded = pad_sequences([encoded], maxlen=max_seq_len-1)
        pred = model.predict(encoded, verbose=0)
        next_char = tokenizer.index_word[np.argmax(pred)]
        seed_text += " "+ next_char
    return seed_text

print(generate_text("hat art thou that vsurp'st this"))
